In [1]:
from sklearn.model_selection import KFold
from tensorflow.keras.applications import ResNet101
from tensorflow.keras import layers, models
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import itertools

# ---- Load and Preprocess Data ----
train_dir = 'D:/Lung_cancer/train'  # Training directory path
target_size = (224, 224)  # Input size for ResNet101
num_classes = 3

# Extract file paths and labels
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=32,  # Temporary batch size
    class_mode='categorical',
    shuffle=False  # To keep track of indices
)
X = np.array(train_generator.filepaths)  # File paths of images
y = np.array(train_generator.classes)    # Corresponding class labels

# ---- Hyperparameter Search Space ----
batch_sizes = [16, 32]
dropout_rates = [0.3, 0.5]
learning_rates = [0.0005, 0.001]
kf = KFold(n_splits=5, shuffle=True, random_state=42)

best_params = {}
best_accuracy = 0

# Iterate over all combinations of hyperparameters
for batch_size, dropout_rate, learning_rate in itertools.product(batch_sizes, dropout_rates, learning_rates):
    print(f"Testing batch_size={batch_size}, dropout={dropout_rate}, lr={learning_rate}")
    
    fold_accuracies = []
    fold_losses = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"\nFold {fold + 1}")
        
        # Split into training and validation sets
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Load data dynamically
        def load_data(file_paths, labels):
            images = []
            for file in file_paths:
                img = load_img(file, target_size=target_size)
                img = img_to_array(img)
                images.append(img)
            images = np.array(images) / 255.0
            labels = np.eye(num_classes)[labels]
            return images, labels
        
        X_train_images, y_train_labels = load_data(X_train, y_train)
        X_val_images, y_val_labels = load_data(X_val, y_val)
        
        # ---- Define and Compile Model ----
        base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
        base_model.trainable = False
        
        x = base_model.output
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dense(512, activation='relu')(x)
        x = layers.Dropout(dropout_rate)(x)
        output = layers.Dense(num_classes, activation='softmax')(x)
        model = models.Model(inputs=base_model.input, outputs=output)
        
        model.compile(
            optimizer=Adam(learning_rate=learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        
        # ---- Train Model ----
        history = model.fit(
            X_train_images, y_train_labels,
            validation_data=(X_val_images, y_val_labels),
            epochs=10,  # Default to 10 epochs
            batch_size=batch_size,
            verbose=1
        )
        
        # ---- Evaluate Model ----
        val_loss, val_accuracy = model.evaluate(X_val_images, y_val_labels, verbose=0)
        print(f"Fold {fold + 1} Accuracy: {val_accuracy * 100:.2f}%")
        print(f"Fold {fold + 1} Loss: {val_loss:.4f}")
        
        fold_accuracies.append(val_accuracy)
        fold_losses.append(val_loss)
    
    # Compute mean accuracy for the current hyperparameter combination
    mean_accuracy = np.mean(fold_accuracies)
    mean_loss = np.mean(fold_losses)
    
    print(f"Mean Accuracy: {mean_accuracy * 100:.2f}%, Mean Loss: {mean_loss:.4f}\n")
    
    # Store best hyperparameters
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_params = {
            "batch_size": batch_size,
            "dropout": dropout_rate,
            "learning_rate": learning_rate,
            "accuracy": best_accuracy
        }

# ---- Final Best Parameters ----
print("Best Parameters:")
print(best_params)


Found 406 images belonging to 3 classes.
Testing batch_size=16, dropout=0.3, lr=0.0005

Fold 1
Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 121s 4s/step - accuracy: 0.3337 - loss: 1.3371 - val_accuracy: 0.4634 - val_loss: 1.0652
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.3173 - loss: 1.2542 - val_accuracy: 0.4634 - val_loss: 1.0869
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.3680 - loss: 1.1013 - val_accuracy: 0.2561 - val_loss: 1.1595
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.3443 - loss: 1.1714 - val_accuracy: 0.4146 - val_loss: 1.0548
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.3604 - loss: 1.1140 - val_accuracy: 0.4024 - val_loss: 1.0687
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 0.4226 - loss: 1.0998 - val_accuracy: 0.4634 - val_loss: 1.0184
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.4048 - loss: 1.1091 - val_accuracy: 0.2561 - val_loss: 1.0875
Epoch 8/10
21/21 ━━━━━